<a href="https://colab.research.google.com/github/mynhungg/Datamining/blob/main/VNTC_Kaggle__MN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
path = "/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle"
os.chdir(path)
!ls

In [3]:
!git clone https://github.com/duyvuleo/VNTC.git

Cloning into 'VNTC'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 160.90 MiB | 7.53 MiB/s, done.
Updating files: 100% (15/15), done.
Filtering content: 100% (2/2), 168.95 MiB | 52.00 MiB/s, done.
fatal: cannot exec '/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/VNTC/.git/hooks/post-checkout': Permission denied


Extract file

In [4]:
!apt install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.6.6-2build1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


In [5]:
!unrar x "/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/VNTC/Data/10Topics/Ver1.1/Test_Full.rar"
!unrar x "/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/VNTC/Data/10Topics/Ver1.1/Train_Full.rar"

!ls

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (55).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (75).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (77).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (85).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (89).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (90).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (91).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (96).txt                       83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (103).txt                      83%  OK 
Extracting  Train_Full/Kinh doanh/KD_ VNE_ (108).txt                      83%  OK 
Extracting  Train_Full/

## Data

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.9 MB/s eta 0:00:00


In [7]:
import os
import torch
from typing import List, Tuple, Dict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [8]:
TRAIN_PATH = "/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full"
TEST_PATH = "/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Test_Full"

In [9]:
category2id = {category: idx for idx, category in enumerate(os.listdir(TRAIN_PATH))}
category2id

{'Chinh tri Xa hoi': 0,
 'Vi tinh': 1,
 'Van hoa': 2,
 'The thao': 3,
 'The gioi': 4,
 'Suc khoe': 5,
 'Phap luat': 6,
 'Kinh doanh': 7,
 'Khoa hoc': 8,
 'Doi song': 9}

In [10]:
data = []
def read_file(category, file_name):
    text_path = os.path.join(TRAIN_PATH, category, file_name)
    with open(text_path, "r", encoding = 'utf-16') as file:
        content = file.read()
    return (content, category2id[category])
print("TRAIN:")
for category in os.listdir(TRAIN_PATH):
    category_path = os.path.join(TRAIN_PATH, category)
    print(category_path)
    data.extend([ read_file(category, file_name) for file_name in os.listdir(category_path)])
test_data = []
print("TEST:")
for category in os.listdir(TEST_PATH):
    category_path = os.path.join(TRAIN_PATH, category)
    print(category_path)
    test_data.extend([ read_file(category, file_name) for file_name in os.listdir(category_path)])

TRAIN:
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Chinh tri Xa hoi
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Vi tinh
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Van hoa
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/The thao
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/The gioi
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Suc khoe
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Phap luat
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Kinh doanh
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Khoa hoc
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Doi song
TEST:
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Kinh doanh
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/Vi tinh
/content/gdrive/MyDrive/Colab Notebooks/VNTC_Kaggle/Train_Full/The thao
/content/gdrive/MyDrive/Colab Notebooks/V

In [11]:
class VNTCDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [12]:
train_data, valid_data = train_test_split(data, test_size = 0.1)

In [13]:
vntc_train_data = VNTCDataset(train_data)
vntc_valid_data = VNTCDataset(valid_data)
vntc_test_data = VNTCDataset(test_data)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  256


In [16]:
def collate_fn(data: List[Tuple[str, int]]):
    texts = []
    labels = []
    for content, label in data:
        texts.append(content)
        labels.append(label)    
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    return input_ids, labels

In [17]:
train_dataloader = torch.utils.data.DataLoader(dataset = vntc_train_data, batch_size = 32, collate_fn = collate_fn, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(dataset = vntc_valid_data, batch_size = 32, collate_fn = collate_fn)
test_dataloader = torch.utils.data.DataLoader(dataset = vntc_test_data, batch_size = 32, collate_fn = collate_fn)

In [18]:
for input_ids, labels in train_dataloader:
    print(input_ids, labels)
    break

tensor([[    0,  3089,    29,  ...,   164,  1820,     2],
        [    0,  1117, 26380,  ...,    11,   563,     2],
        [    0,  1696,  1624,  ..., 30939,     4,     2],
        ...,
        [    0, 51899,    93,  ...,     6,    91,     2],
        [    0,   680,   413,  ...,    73,   238,     2],
        [    0,    92,    16,  ..., 18232, 10557,     2]]) tensor([0, 0, 8, 0, 3, 6, 1, 8, 1, 9, 7, 4, 6, 0, 3, 5, 5, 0, 1, 5, 5, 6, 3, 3,
        6, 1, 5, 0, 3, 4, 8, 5])


## Model

In [19]:
class PhobertForNewsClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = num_classes)
        for param in self.phobert.roberta.parameters():
            param.requires_grad = False
    def forward(self, input_ids, labels):
        out = self.phobert(input_ids, labels = labels)
        probs = torch.softmax(out['logits'], dim = -1)
        preds = torch.argmax(probs, dim = -1).cpu().detach().numpy()
        out['preds'] = preds
        return out

In [20]:
model = PhobertForNewsClassification(len(category2id))

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [21]:
def get_num_parameters(module: torch.nn.Module) -> Dict[str, int]:
    """

    Get the number of trainable, fixed and total parameters of a pytorch module.

    :param module: pytorch module
    :return: dict containing number of parameters
    """
    trainable = 0
    fixed = 0
    for p in module.parameters():
        if p.requires_grad:
            trainable += p.numel()
        else:
            fixed += p.numel()
    return {"trainable": trainable, "fixed": fixed, "total": trainable + fixed}

In [22]:
get_num_parameters(model)

{'trainable': 598282, 'fixed': 134407680, 'total': 135005962}

## Train

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [24]:
LR = 1e-4
EPOCH = 5
LOG_STEP = 100

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr= LR)

In [26]:
print(f"Train Loader Step: {len(train_dataloader)}")
print(f"Valid Loader Step: {len(valid_dataloader)}")

Train Loader Step: 950
Valid Loader Step: 106


In [27]:
model = model.to(device)

In [28]:
losses = 0
for i in range(EPOCH):
    print(f"Epoch {i}:")
    model.train()
    for step, (input_ids, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        out = model(input_ids, labels)
        loss = out['loss']
        loss.backward()
        optimizer.step()
        losses += loss.detach().cpu().item()
        if step % LOG_STEP == 0 and step > 0:
            print(f"Step {step}, Avg Loss {loss}")
            losses = 0
    
    preds = []
    trues = []
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(valid_dataloader):
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            out = model(input_ids, labels)
            loss = out['loss']
            valid_loss += loss.detach().cpu().item()
            preds.extend(out['preds'])
            trues.extend(labels.detach().cpu().numpy())
    print(classification_report(trues, preds, target_names = list(category2id.keys())))
    print(f"Valid loss: {valid_loss / len(valid_dataloader)}")

Epoch 0:
Step 100, Avg Loss 1.582879662513733
Step 200, Avg Loss 1.2644597291946411
Step 300, Avg Loss 1.130757212638855
Step 400, Avg Loss 0.9819436073303223
Step 500, Avg Loss 0.6252636909484863
Step 600, Avg Loss 0.8310559988021851
Step 700, Avg Loss 0.610502302646637
Step 800, Avg Loss 0.7264055013656616
Step 900, Avg Loss 0.6699568629264832
                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.66      0.75      0.70       533
         Vi tinh       0.93      0.72      0.81       262
         Van hoa       0.85      0.78      0.81       319
        The thao       0.97      0.93      0.95       542
        The gioi       0.83      0.79      0.81       307
        Suc khoe       0.75      0.85      0.79       305
       Phap luat       0.69      0.83      0.75       364
      Kinh doanh       0.79      0.74      0.76       236
        Khoa hoc       0.83      0.59      0.69       186
        Doi song       0.78      0.78      0.78       322

        accu

## Test

In [29]:
preds = []
trues = []
model.eval()
with torch.no_grad():
    for step, (input_ids, labels) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        out = model(input_ids, labels)
        loss = out['loss']
        preds.extend(out['preds'])
        trues.extend(labels.detach().cpu().numpy())
        
print(classification_report(trues, preds, target_names = list(category2id.keys())))

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.75      0.72      0.73      5219
         Vi tinh       0.94      0.81      0.87      2481
         Van hoa       0.90      0.84      0.87      3080
        The thao       0.98      0.95      0.96      5298
        The gioi       0.89      0.86      0.87      2898
        Suc khoe       0.78      0.89      0.83      3384
       Phap luat       0.72      0.88      0.79      3868
      Kinh doanh       0.83      0.75      0.79      2552
        Khoa hoc       0.83      0.72      0.77      1820
        Doi song       0.80      0.82      0.81      3159

        accuracy                           0.83     33759
       macro avg       0.84      0.83      0.83     33759
    weighted avg       0.84      0.83      0.83     33759

